In [87]:
import usaddress
import pandas as pd
import numpy as np
import collections

In [88]:
df = pd.read_excel('Test Data Texas2.xlsx')
df


,order date,entry person,old cust nbr,customer name,order type,invoice type,ship date,cap,division,ship mer amt,ship tax,ship to address1,ship to address2,ship to city,ship to state,ship to zip,Zip 5,County
0,2024-01-23,API,8802.0,AMAZON PS - LOC 2,CHG,SI,2024-01-23,1,12,21.77,0.00,353 anderson county road 2139,NaN,Palestine,TX,75801,75801,Anderson
1,2024-01-25,API,8813.0,AMAZON PS - LOC 4,CHG,SI,2024-01-26,1,12,116.18,0.00,1510 Fm 3224,NaN,Palestine,TX,75803-2169,75803,Anderson
2,2024-01-26,API,9934.0,WALMART MARKETPLACE PS LOC 4,CHG,SI,2024-01-29,1,12,45.96,0.00,2301 Country Club Rd,NaN,Palestine,TX,75803-8339,75803,Anderson
3,2024-01-22,API,8813.0,AMAZON PS - LOC 4,CHG,SI,2024-01-22,1,12,8.09,0.00,5765 E State Highway 294,NaN,Elkhart,TX,75839-3135,75839,Anderson
4,2024-01-23,INTERNET,NaN,C Jett oil tools,CASH,SI,2024-01-24,2,12,774.00,48.38,Attn: Corey Bruton,2277 Se 2800,Andrews,TX,79714-5771,79714,Andrews
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,2024-01-09,API,8802.0,AMAZON PS - LOC 2,CHG,SI,2024-01-10,1,12,27.11,0.00,13116 Coriander Dr,NaN,Austin,TX,78729-6472,78729,Williamson
777,2024-01-13,INTERNET,NaN,Xavier Granados,CASH,SI,2024-01-15,2,12,58.96,4.86,109 N Tranquility Dr,NaN,La Vernia,TX,78121-5504,78121,Wilson
778,2024-01-02,EDI,9570.0,ZORO TOOLS PS - LOC 5,CHG,SI,2024-01-03,2,12,69.60,0.00,Buffie Pryor,1870 Illinois St,Rhome,TX,76078,76078,Wise
779,2024-01-23,INTERNET,NaN,A-1Escort Ser.,CASH,SI,2024-01-24,2,12,391.00,26.39,Attn: Randy Warnell,109 Private Road 1309,Bridgeport,TX,76426-6207,76426,Wise


In [120]:
list_add = []
j = []
for i in range(len(df)):
    list_add.append(str(df['ship to address1'].iloc[i]) + ', ' + str(df['ship to address2'].fillna('').iloc[i]))
    try:
        j.append(usaddress.tag(list_add[-1])[0])
    except:
        j.append(collections.OrderedDict([('Recipient','bad')]))

new_df = pd.DataFrame(j)
nans = ~new_df['AddressNumber'].isna()
new_df=new_df[nans]
new_df.fillna('', inplace=True)
df['Street Address'] = (new_df['AddressNumber'] + ' ' + new_df['StreetNamePreType'] + ' ' + 
        new_df['StreetNamePreDirectional'] + ' ' + new_df['StreetName'] + ' ' + 
        new_df['StreetNamePostType'] + ' ' + new_df['StreetNamePostDirectional'])
lookup_list = {}
debug = df[['Street Address', 'ship to city', 'ship to zip']][nans].drop_duplicates()
for item in debug.to_numpy():
    key = item[0]  # Extracting the first part of the address as the key
    lookup_list[key] = tuple(item)

In [92]:
new_df.head(10)

,AddressNumber,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePreDirectional,Recipient,OccupancyType,StreetNamePostDirectional,OccupancyIdentifier,SubaddressIdentifier,BuildingName,StateName,ZipCode,PlaceName,SubaddressType,USPSBoxType,USPSBoxID
0,353,anderson county road,2139,,,,,,,,,,,,,,
1,1510,Fm,3224,,,,,,,,,,,,,,
2,2301,,Country Club,Rd,,,,,,,,,,,,,
3,5765,State Highway,294,,E,,,,,,,,,,,,
4,2277,,2800,,Se,Attn: Corey Bruton,,,,,,,,,,,
5,1613,,3101,,Se,,,,,,,,,,,,
6,907,,PERISHING,,,,,,,,,,,,,,
11,14151,Fm,2790,,W,,,S,,,,,,,,,
12,817,,Bryant,St,N,,Ste,,C,,,,,,,,
13,1324,,Kloecker,Rd,,Attn: Jennifer Sullivan,,,,,,,,,,,


In [121]:
lookup_list

{'353 anderson county road  2139  ': ('353 anderson county road  2139  ',
  'Palestine',
  75801),
 '1510 Fm  3224  ': ('1510 Fm  3224  ', 'Palestine', '75803-2169'),
 '2301   Country Club Rd ': ('2301   Country Club Rd ',
  'Palestine',
  '75803-8339'),
 '5765 State Highway E 294  ': ('5765 State Highway E 294  ',
  'Elkhart',
  '75839-3135'),
 '2277  Se 2800  ': ('2277  Se 2800  ', 'Andrews', '79714-5771'),
 '1613  Se 3101  ': ('1613  Se 3101  ', 'Andrews', '79714-5835'),
 '907   PERISHING  ': ('907   PERISHING  ', 'LUFKIN', 75904),
 '14151 Fm W 2790  S': ('14151 Fm W 2790  S', 'Lytle', 78052),
 '817  N Bryant St ': ('817  N Bryant St ', 'Pleasanton', '78064-2515'),
 '1324   Kloecker Rd ': ('1324   Kloecker Rd ', 'Sealy', '77474-9337'),
 '594   Upper Moffett Rd ': ('594   Upper Moffett Rd ',
  'Medina',
  '78055-3951'),
 '660 Loop Nw 230  ': ('660 Loop Nw 230  ', 'Smithville', 78957),
 '386 Highway  304  ': ('386 Highway  304  ', 'Bastrop', 786023249),
 '4754 Fm  535  ': ('4754 Fm  5

In [113]:
new_df.iloc[7]

AddressNumber                14151
StreetNamePreType               Fm
StreetName                    2790
StreetNamePostType                
StreetNamePreDirectional         W
Recipient                         
OccupancyType                     
StreetNamePostDirectional        S
OccupancyIdentifier               
SubaddressIdentifier              
BuildingName                      
StateName                         
ZipCode                           
PlaceName                         
SubaddressType                    
USPSBoxType                       
USPSBoxID                         
Name: 11, dtype: object

In [ ]:
list_add = []
j = []
for i in range(len(df)):
    list_add.append(str(df['ship to address1'].iloc[i]) + ', ' + str(df['ship to address2'].fillna('').iloc[i]))
    try:
        j.append(usaddress.tag(list_add[-1])[0])
    except:
        j.append(collections.OrderedDict([('Recipient','bad')]))

new_df = pd.DataFrame(j)
nans = ~new_df['AddressNumber'].isna()
new_df=new_df[nans]
new_df.fillna('', inplace=True)
df['Street Address'] = (new_df['AddressNumber'] + ' ' + new_df['StreetNamePreType'] + ' ' + 
        new_df['StreetNamePreDirectional'] + ' ' + new_df['StreetName'] + ' ' + 
        new_df['StreetNamePostType'] + ' ' + new_df['StreetNamePostDirectional'])
lookup_list = {}
debug = df[['Street Address', 'ship to city', 'ship to zip']].drop_duplicates()
for item in debug.to_numpy():
    key = item[0]  # Extracting the first part of the address as the key
    lookup_list[key] = tuple(item)

In [78]:
df['ship to address2'].fillna('')

0                           
1                           
2                           
3                           
4               2277 Se 2800
               ...          
776                         
777                         
778         1870 Illinois St
779    109 Private Road 1309
780                         
Name: ship to address2, Length: 781, dtype: object

In [79]:
test_list = []
for i in range(len(df)):
    test_list.append(str(df['ship to address1'].iloc[i]) + ', ' + str(df['ship to address2'].fillna('').iloc[i]))

In [61]:
list_add = []
j = []
for i in range(len(df)):
    list_add.append(str(df['ship to address1'].iloc[i]) + ', ' + str(df['ship to address2'].fillna('').iloc[i]))
    try:
        j.append(usaddress.tag(list_add[-1])[0])
    except:
        j.append(collections.OrderedDict([('Recipient','bad')]))
new_df = pd.DataFrame(j)
new_df.head(10)

,AddressNumber,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePreDirectional,Recipient,OccupancyType,StreetNamePostDirectional,OccupancyIdentifier,SubaddressIdentifier,BuildingName,StateName,ZipCode,PlaceName,SubaddressType,USPSBoxType,USPSBoxID
0,353,anderson county road,2139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1510,Fm,3224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2301,NaN,Country Club,Rd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5765,State Highway,294,NaN,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2277,NaN,2800,NaN,Se,Attn: Corey Bruton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1613,NaN,3101,NaN,Se,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,907,NaN,PERISHING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,XXXXX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,XXXXX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,XXXXX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
nans = ~new_df['AddressNumber'].isna()
for i in range(len(new_df[~nans])):
    print(new_df[~nans].iloc[i].values)

[nan nan nan nan nan nan 'XXXXX' nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan 'XXXXX' nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan 'XXXXX' nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan 'XXXXX' nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan 'XXXXX' nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan 'XXXXX' nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan nan nan '6134' nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan 'XXXXX' nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan 'XXXXX' nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan 'XXXXX' nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan 'XXXXX' nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan 'XXXXX' nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan 'XXXXX' nan nan nan nan nan nan nan nan nan nan]
[nan nan nan nan nan nan 'XXXXX' nan na

In [82]:
new_df=new_df[nans]
new_df.fillna('', inplace=True)
new_df

C:\Users\miles\AppData\Local\Temp\ipykernel_12348\3671672500.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df=new_df[nans]


,AddressNumber,StreetNamePreType,StreetName,StreetNamePostType,StreetNamePreDirectional,Recipient,OccupancyType,StreetNamePostDirectional,OccupancyIdentifier,SubaddressIdentifier,BuildingName,StateName,ZipCode,PlaceName,SubaddressType,USPSBoxType,USPSBoxID
0,353,anderson county road,2139,,,,,,,,,,,,,,
1,1510,Fm,3224,,,,,,,,,,,,,,
2,2301,,Country Club,Rd,,,,,,,,,,,,,
3,5765,State Highway,294,,E,,,,,,,,,,,,
4,2277,,2800,,Se,Attn: Corey Bruton,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,13116,,Coriander,Dr,,,,,,,,,,,,,
777,109,,Tranquility,Dr,N,,,,,,,,,,,,
778,1870,,Illinois,St,,Buffie Pryor,,,,,,,,,,,
779,109,,Private,Road,,Attn: Randy Warnell,,,1309,,,,,,,,


In [45]:
ordered_columns = ['AddressNumber', 'StreetNamePreType', 'StreetNamePreDirectional', 'StreetName', 'StreetNamePostType']

In [86]:
x = ('12'+'34'+'56')
x

'123456'

In [84]:
temp = new_df['AddressNumber'] + ' ' + new_df['StreetNamePreType'] + ' ' + new_df['StreetNamePreDirectional'] + ' ' + new_df['StreetName'] + ' ' + new_df['StreetNamePostType'] + ' ' + new_df['StreetNamePostDirectional']
list_lookup = []
for address in temp:
    list_lookup.append(address)
list_lookup

['353 anderson county road  2139  ',
 '1510 Fm  3224  ',
 '2301   Country Club Rd ',
 '5765 State Highway E 294  ',
 '2277  Se 2800  ',
 '1613  Se 3101  ',
 '907   PERISHING  ',
 '14151 Fm W 2790  S',
 '817  N Bryant St ',
 '1324   Kloecker Rd ',
 '594   Upper Moffett Rd ',
 '594   Upper Moffett Rd ',
 '660 Loop Nw 230  ',
 '386 Highway  304  ',
 '4754 Fm  535  ',
 '1442   Old Mcdade Rd ',
 '3712  E VETERANS MEMORIAL BLVD ',
 '1226 Us Hwy East 190  ',
 '1822   Duce Spur ',
 '710   Swanner Loop ',
 '12502   Salado Springs Cir ',
 '1333   Buena Vista St ',
 '6100   Bandera Road ',
 '8802   Tradewind Dr ',
 '5757   Farinon Dr ',
 '8033   Orchard Bend St ',
 '8033   Orchard Bend St ',
 '20770 Hwy  281  N',
 '2715   Running Fawn  ',
 '10018   Ramblin River Rd ',
 '10025   Shepherd Rd ',
 '16321   Shepherd Rd ',
 '12980   Interstate 35  S',
 '4229   Fort Palmer Blvd ',
 '1039  W Hildebrand Ave ',
 '1401   Patricia  ',
 '922   Isom Rd ',
 '5230   Eisenhauer Rd ',
 '3453  N Panam Expy ',
 '463

In [33]:
len(temp[~temp.isna()])

591

In [27]:
for address in temp[~temp.isna()]:
    print(address)

2301 Country Club Rd
817 Bryant St
1324 Kloecker Rd
594 Upper Moffett Rd
594 Upper Moffett Rd
1442 Old Mcdade Rd
3712 VETERANS MEMORIAL BLVD
1822 Duce Spur
710 Swanner Loop
12502 Salado Springs Cir
1333 Buena Vista St
6100 Bandera Road
8802 Tradewind Dr
5757 Farinon Dr
8033 Orchard Bend St
8033 Orchard Bend St
10018 Ramblin River Rd
10025 Shepherd Rd
16321 Shepherd Rd
4229 Fort Palmer Blvd
1039 Hildebrand Ave
922 Isom Rd
5230 Eisenhauer Rd
3453 Panam Expy
4634 Presa St
7500 Callaghan Rd
3003 Old Elm Way
3111 Quakertown Dr
5815 Rim Pass
6305 Camp Bullis Rd
6305 Camp Bullis Rd
1534 Clementson Dr
214 Ej Ct
4602 Woodstock Ln
3730 Main St.
17206 Pearland Sites Rd
1001 Main St
2132 Country Club Dr
4344 Main St
4344 Main St
3706 Sheldon Dr
1502 HARVEY RD
1502 HARVEY RD
124 Cinnamon Loop
126 Foxwood Trl
411 Lariat Ln
1500 Gulf Boulevard
292 Kings Hwy
724 Pine Falls Dr
1631 Lincoln St
28051 Mile 3 Rd
11419 Sunnyside Dr
535 Leblanc St
732 San Antonio St
2100 Purdue St
6913 K Ave
2803 Skyview Dr
